In [ ]:
import os
import pandas as pd
import numpy as np

import plotly.graph_objects as go

In [ ]:
data_folder = os.path.join('..', '..', 'data', 'tarragona')

clean_data_folder = os.path.join(data_folder, 'clean_data')
solar_radiation_folder = os.path.join(data_folder, 'solar_radiation')

projections_folder = os.path.join(data_folder, 'future_projections')
my_projections_folder = os.path.join(projections_folder, 'found_by_me')
cat_projections_folder = os.path.join(projections_folder, 'cat')

# Load Data

In [ ]:
solar_df = pd.read_csv(os.path.join(solar_radiation_folder, 'daily_solar_radiation_Guiamets_2008_2024.csv'), sep=';', decimal=',')

In [ ]:
solar_df

In [ ]:
# drop rows that have three spaces in at least one of the columns
solar_df = solar_df[~solar_df['Date'].str.contains('   ', na=False)]
solar_df = solar_df[~solar_df['Date (maximum value)'].str.contains('   ', na=False)]
solar_df = solar_df[~solar_df['Mean'].str.contains('   ', na=False)]
solar_df = solar_df[~solar_df['Maximum'].str.contains('   ', na=False)]

In [ ]:
solar_df['Date'] = pd.to_datetime(solar_df['Date'], format='%d/%m/%Y %H:%M')
solar_df['Date (maximum value)'] = pd.to_datetime(solar_df['Date (maximum value)'], format='%d/%m/%Y %H:%M')

solar_df['Mean'] = solar_df['Mean'].replace(',', '.', regex=True).astype(float)
solar_df['Maximum'] = solar_df['Maximum'].replace(',', '.', regex=True).astype(float)

In [ ]:
solar_df['Date'] = solar_df['Date'].dt.date
solar_df['Date (maximum value)'] = solar_df['Date (maximum value)'].dt.date

In [ ]:
mean_solar_df = solar_df[['Date', 'Mean']]
max_solar_df = solar_df[['Date (maximum value)', 'Maximum']]

mean_solar_df = mean_solar_df.rename(columns={'Date': 'DateTime', 'Mean': 'Solar Radiation (W/m^2)'})
max_solar_df = max_solar_df.rename(columns={'Date (maximum value)': 'DateTime', 'Maximum': 'Solar Radiation (W/m^2)'})

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=mean_solar_df['DateTime'], y=mean_solar_df['Solar Radiation (W/m^2)'], mode='lines', name='Mean Solar Radiation (W/m^2)'))

fig.update_layout(title='Mean Solar Radiation in Guiamets (2008-2024)',
                     xaxis_title='Date',
                     yaxis_title='Solar Radiation (W/m^2)')

fig.show()

In [ ]:
mean_solar_df.set_index('DateTime', inplace=True)
max_solar_df.set_index('DateTime', inplace=True)

mean_solar_df.index = pd.to_datetime(mean_solar_df.index)
max_solar_df.index = pd.to_datetime(max_solar_df.index)

mean_solar_df = mean_solar_df.resample('D').mean()
max_solar_df = max_solar_df.resample('D').mean()

# Inspect Data

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=mean_solar_df.index, y=mean_solar_df['Solar Radiation (W/m^2)'], mode='lines', name='Mean solar radiation'))
# fig.add_trace(go.Scatter(x=max_solar_df.index, y=max_solar_df['Solar Radiation (W/m^2)'], mode='lines', name='Maximum solar radiation'))

fig.update_layout(
    title='Daily solar radiation in Guiamets (2008-2024)',
    xaxis_title='Date',
    yaxis_title='Solar radiation (W/m²)'
)

fig.show()

In [ ]:
# resample to monthly

mean_solar_df = mean_solar_df.resample('M').mean()
max_solar_df = max_solar_df.resample('M').mean()

fig = go.Figure()

fig.add_trace(go.Scatter(x=mean_solar_df.index, y=mean_solar_df['Solar Radiation (W/m^2)'], mode='lines', name='Mean solar radiation'))
# fig.add_trace(go.Scatter(x=max_solar_df.index, y=max_solar_df['Solar Radiation (W/m^2)'], mode='lines', name='Maximum solar radiation'))

fig.update_layout(
    title='Monthly solar radiation in Guiamets (2008-2024)',
    xaxis_title='Date',
    yaxis_title='Solar radiation (W/m²)'
)

fig.show()

# Projections

In [ ]:
daily_solar_df = pd.read_excel(os.path.join(my_projections_folder, "solar_radiation.xlsx"))

In [ ]:
random_solar_df = pd.read_excel(os.path.join(cat_projections_folder, "solar_radiation_random.xlsx"))

In [ ]:
daily_solar_df.info()

In [ ]:
random_solar_df.info()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=daily_solar_df['DateTime'], y=daily_solar_df['Solar Radiation (W/m²)'], mode='lines', name='Mean solar radiation (PoliMi)'))

fig.add_trace(go.Scatter(x=mean_solar_df.index, y=mean_solar_df['Solar Radiation (W/m^2)'], mode='lines', name='Mean solar radiation (observed)'))

fig.add_trace(go.Scatter(x=random_solar_df['time'], y=random_solar_df['rsdt'], mode='lines', name='Mean solar radiation (Cat)'))

fig.update_layout(
    title='Solar Radiation RCP 8.5',
    xaxis_title='Date',
    yaxis_title='Solar radiation (W/m²)'
)

fig.show()

# Store Data

In [ ]:
mean_solar_df.to_excel(
    os.path.join(clean_data_folder, 'solar_radiation.xlsx')
)